In [1]:
import pandas as pd
import os
import numpy as np
import cv2

In [2]:
class4_bbox_path = 'C:\\Users\\dental-1\\Documents\\dent_seg\\8_train_yolo_four_class_tooth_segmentation\\'+\
                    'yolov5\\runs\\detect\\exp2\\labels'

In [3]:
file_save_path = 'C:\\Users\\dental-1\\dent_seg\\1_helper_code\\overlapping1\\'

BBOX_PATH = class4_bbox_path

In [4]:
bounding_boxes = os.listdir(BBOX_PATH)
bounding_boxes[:5]

['.ipynb_checkpoints',
 '1.2.840.114257.1.1637304243952960394983328788351918369.txt',
 '1.2.840.114257.3.6.10028842080465500458430592052923973032886.txt',
 '1.2.840.114257.3.6.10030708032351340458422440040212943937371.txt',
 '1.2.840.114257.3.6.10042008176214120458429902067895443955409.txt']

In [5]:
bad_path = 'C:\\Users\\dental-1\\dent_seg\\1_helper_code\\overlapping_bboxes'
bad_bbox = os.listdir('overlapping_bboxes')

len(bad_bbox)

1072

takes all detected labels from 4 class model and if the label file is not in overlapping_bboxes/, then save it to good_bboxes/

In [28]:
pwd

'C:\\Users\\dental-1\\Documents\\dent_seg\\1_helper_code'

In [8]:
for bb in bounding_boxes:
    if 'txt' in bb and bb not in bad_bbox:
        filename = BBOX_PATH+'\\' + bb
        df = pd.read_csv(filename, sep=' ', header=None)
        bad_df_to_write.to_csv('good_bboxes\\'+bb, index=False, header=False, sep=' ')

In [7]:
counter = []
for bb in bounding_boxes:
    if 'txt' in bb: #and bb not in bad_bbox:
        filename = BBOX_PATH+'\\' + bb
        # get df
        df = pd.read_csv(filename, sep=' ', header=None)
        df.rename(columns={0: 'class', 1: 'x_center', 2: 'y_center', 3: 'height', 4: 'width', 5: 'confidence'}, inplace=True)
        df.drop(columns=['confidence'], inplace=True)
        # get top/bottom
        average_y = df['y_center'].mean()
        df['top/bottom'] = np.where(df.y_center < average_y, 'top', 'bottom')
        # sort teeth
        df.sort_values(by=['top/bottom', 'x_center'], inplace=True)
        # drop top/bottom column
        df.drop(columns=['top/bottom'], inplace=True)

        
        bad_df_arr = []
        count = 0
        THRESH = 0.005
        # if (previous and current values are <= 0.005 difference or equal to each other) and 
        #     (class labels are the same): save
        # uncomment df.to_csv at bottom to save
        for idx in range(1,len(df)):
            if (df['class'].iloc[idx-1]!=df['class'].iloc[idx]):
                if ((abs(df['x_center'].iloc[idx-1] - df['x_center'].iloc[idx])<=THRESH or df['x_center'].iloc[idx-1]==df['x_center'].iloc[idx]) and
                (abs(df['y_center'].iloc[idx-1] - df['y_center'].iloc[idx])<=THRESH or df['y_center'].iloc[idx-1]==df['y_center'].iloc[idx]) and
                (abs(df['width'].iloc[idx-1] - df['width'].iloc[idx])<=THRESH or df['width'].iloc[idx-1]==df['width'].iloc[idx]) and
                (abs(df['height'].iloc[idx-1] - df['height'].iloc[idx])<=THRESH or df['height'].iloc[idx-1]==df['height'].iloc[idx])):
                    #counter += 1
                    dat0 = {'class' : df['class'].iloc[idx-1],
                           'x_center' : df['x_center'].iloc[idx-1],
                           'y_center' : df['y_center'].iloc[idx-1],
                           'height' : df['height'].iloc[idx-1],
                           'width' : df['width'].iloc[idx-1]}
                    dat1 = {'class' : df['class'].iloc[idx],
                           'x_center' : df['x_center'].iloc[idx],
                           'y_center' : df['y_center'].iloc[idx],
                           'height' : df['height'].iloc[idx],
                           'width' : df['width'].iloc[idx]}
                    bad_df_arr.append(dat0)
                    bad_df_arr.append(dat1)
                    count+=1
        #print(count)
        if count > 0:
            #print(count)
            bad_df_to_write = pd.DataFrame(bad_df_arr)
            save_name = file_save_path + bb
            bad_df_to_write.to_csv('overlapping1\\'+bb, index=False, header=False, sep=' ')
        counter.append(count)
        
print('len data: '+str(len(counter)))
avg_bad = len([x for x in counter if x > 0])/len(counter)
print('proportion of images having at least 1 bad/closely overlapping bounding box: '+str(avg_bad))

len data: 2287
proportion of images having at least 1 bad/closely overlapping bounding box: 0.5360734586794927


In [24]:
# 3 class detections- 40% of images have at least 1 bad/closely overlapping bounding box
len([x for x in counter if x > 0]) / len(counter)

0.40342960288808666

In [25]:
len(counter)

2216

In [11]:
# 4 class detections- 53% of images have at least 1 bad/closely overlapping bounding box
len([x for x in counter if x > 0]) / len(counter)

0.5362762237762237

In [7]:
len(counter)

2288